In [1]:
import pandas as pd
import pcprutils as ut
import numpy as np
import altair as alt

In [9]:
hn_reg = pd.read_csv("hnscc_regression.tsv", sep="\t")
hn_reg

,Tissue,Gene,m,b
0,Normal,AADAC,0.599738,17.448712
1,Normal,AAK1,-0.004513,24.396185
2,Normal,AARS2,0.045875,23.694298
3,Normal,AARSD1,1.581359,10.203151
4,Normal,AASDHPPT,0.166157,23.169844
...,...,...,...,...
6083,Tumor,ZSWIM8,0.342667,20.922590
6084,Tumor,ZW10,0.313533,21.579339
6085,Tumor,ZWINT,0.992880,8.764980
6086,Tumor,ZXDC,2.057887,-2.979253


In [3]:
prot_trans = ut.load_prot_trans([
    "ccrcc",
    "endometrial",
    "hnscc",
    "lscc",
    "luad",
])

cptac warning: Due to dropping the specified levels, dataframe now has 907 duplicated column headers. (/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pcprutils/__init__.py, line 27)
cptac warning: Due to dropping the specified levels, dataframe now has 907 duplicated column headers. (/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pcprutils/__init__.py, line 41)


cptac warning: The LSCC data is currently strictly reserved for CPTAC investigators. Otherwise, you are not authorized to access these data. Additionally, even after these data become publicly available, they will be subject to a publication embargo (see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details). (/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pcprutils/__init__.py, line 163)
cptac warning: Due to dropping the specified levels, dataframe now has 858 duplicated column headers. (/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pcprutils/__init__.py, line 27)
cptac warning: Due to dropping the specified levels, dataframe now has 858 duplicated column headers. (/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pcprutils/__init__.py, line 41)


cptac warning: Due to dropping the specified levels, dataframe now has 745 duplicated column headers. (/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pcprutils/__init__.py, line 27)
cptac warning: Due to dropping the specified levels, dataframe now has 745 duplicated column headers. (/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pcprutils/__init__.py, line 41)


In [8]:
hn_prot_trans = prot_trans["hnscc"]

In [13]:
def get_resid(reg_df, prot_trans_df):
    joined = prot_trans_df.join(reg_df, on="Gene", how="inner")
    
    return joined
    
    x = hn_prot_trans["Transcriptomics"]
    y = hn_prot_trans["Proteomics"]
    m = hn_reg["m"]
    b = hn_reg["b"]
    
    orth_m = -1 / m
    orth_b = y - orth_m * x
    int_x = (b - orth_b) / (orth_m - m)
    int_y = m * int_x + b
    
    d = np.sqrt((int_x - x) ** 2 + (int_y - y) ** 2)
    
    return pd.DataFrame({
        
    })

get_resid(hn_reg, hn_prot_trans)

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

In [14]:
hn_reg.dtypes

Tissue     object
Gene       object
m         float64
b         float64
dtype: object

In [15]:
hn_prot_trans.dtypes

Patient_ID          object
Gene                object
Proteomics         float64
Tissue              object
Transcriptomics    float64
dtype: object